<a href="https://colab.research.google.com/github/birdx0810/LearningTensorFlow/blob/main/TensorFlow___vs_PyTorch_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import os
import random

from tqdm import trange
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Set random seed
os.environ['PYTHONHASHSEED'] = str(42)
random.seed(42)
np.random.seed(42)
tf.compat.v1.set_random_seed(42)

with open("sample_data/anscombe.json", "r") as f:
  tmp = json.load(f)

d1 = []
d2 = []
d3 = []
d4 = []

for d in tmp:
  if d['Series'] == 'I':
    d1.append([d["X"], d["Y"]])
  if d['Series'] == 'II':
    d2.append([d["X"], d["Y"]])
  if d['Series'] == 'III':
    d3.append([d["X"], d["Y"]])
  if d['Series'] == 'IV':
    d4.append([d["X"], d["Y"]])

data = np.concatenate([[d1], [d2], [d3], [d4]])
print(data.shape)

(4, 11, 2)


In [3]:
learning_rate = 1e-3
epochs = 10
batch_size = 2

num_layers = 2
max_seq_len = 11
input_dim = 2
hidden_dim = 10

In [4]:
def get_batch(x, iteration, batch_size):
  time = [len(x[i][:,0]) for i in range(len(x))]

  start = iteration * batch_size
  end = start + batch_size

  x_mb = x[start:end]
  t_mb = time[start:end]

  return x_mb, t_mb

## TensorFlow 1.0


In [5]:
enc_rnn_cell = tf.nn.rnn_cell.GRUCell(num_units=hidden_dim, activation=tf.nn.tanh)
dec_rnn_cell = tf.nn.rnn_cell.GRUCell(num_units=input_dim, activation=tf.nn.tanh)

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.


In [6]:
def model(data):
  tf.reset_default_graph()

  # Input place holders
  X = tf.placeholder(tf.float32, [None, max_seq_len, input_dim])
  S = tf.placeholder(tf.float32, [None])

  def encoder (X, S):
    with tf.variable_scope("encoder", reuse=tf.AUTO_REUSE):

      e_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.GRUCell(num_units=hidden_dim, activation=tf.nn.tanh) for _ in range(num_layers)])

      e_outputs, e_last_states = tf.nn.dynamic_rnn(e_cell, X, dtype=tf.float32, sequence_length=S)

      H = tf.layers.dense(e_outputs, hidden_dim, activation=tf.nn.sigmoid)
    
    return H
        
  def decoder (H, S):
    with tf.variable_scope("decoder", reuse=tf.AUTO_REUSE):

      r_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.GRUCell(num_units=input_dim, activation=tf.nn.tanh) for _ in range(num_layers)])

      r_outputs, r_last_states = tf.nn.dynamic_rnn(r_cell, H, dtype=tf.float32, sequence_length=S)

      X_tilde = tf.layers.dense(r_outputs, input_dim, activation=None)
    
    return X_tilde

  H = encoder(X, S)
  X_tilde = decoder(H, S)

  loss = tf.losses.mean_squared_error(X, X_tilde)

  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

  # Initializing the variables 
  init_global = tf.global_variables_initializer()
  init_local = tf.local_variables_initializer()

  # Launch the graph 
  with tf.Session() as sess:
    sess.run(init_global)
    sess.run(init_local)
    
    # Training cycle
    for epoch in range(epochs):
      total_loss = 0.
      iterations = int(len(data)/batch_size)

      for i in trange(iterations):
        # Get mini batch
        batch_xs, batch_ts = get_batch(data, i, batch_size)
        
        # Fit training using batch data 
        _, batch_loss = sess.run([optimizer, loss], feed_dict={X: batch_xs, S: batch_ts})

        total_loss += batch_loss/iterations
      
      print(f"\nEpoch: {epoch}\tLoss: {total_loss}")
      


In [7]:
model(data)

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


  0%|          | 0/2 [00:00<?, ?it/s]


Epoch: 0	Loss: 74.75547790527344

Epoch: 1	Loss: 74.64163970947266

Epoch: 2	Loss: 74.52843856811523

Epoch: 3	Loss: 74.41592407226562

Epoch: 4	Loss: 74.30418014526367

Epoch: 5	Loss: 74.19321823120117

Epoch: 6	Loss: 74.0830078125

Epoch: 7	Loss: 73.97346115112305


100%|██████████| 2/2 [00:00<00:00, 62.78it/s]



Epoch: 8	Loss: 73.86439895629883

Epoch: 9	Loss: 73.75564575195312


## PyTorch

In [8]:
import torch

os.environ['PYTHONHASHSEED'] = str(42)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


In [9]:
class Encoder(torch.nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.emb_rnn = torch.nn.GRU(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
    self.emb_linear = torch.nn.Linear(hidden_dim, hidden_dim)
    self.emb_sigmoid = torch.nn.Sigmoid()

  def forward(self, X):
    H_o, H_t = self.emb_rnn(X)
    logits = self.emb_linear(H_o)
    H = self.emb_sigmoid(logits)
    return H

class Decoder(torch.nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    self.rec_rnn = torch.nn.GRU(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
    self.rec_linear = torch.nn.Linear(hidden_dim, input_dim)

  def forward(self, H):
    H_o, H_t = self.rec_rnn(H)
    X_tilde = self.rec_linear(H_o)
    return X_tilde

class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.encoder = Encoder()
    self.decoder = Decoder()
    self.criterion = torch.nn.MSELoss()

  def forward(self, X):
    H = self.encoder(X)
    X_tilde = self.decoder(H)
    loss = self.criterion(X_tilde, X)
    return loss

In [11]:
model = Model()
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
  iterations = int(len(data)/batch_size)
  for i in trange(iterations):
    model.zero_grad()
    # Get mini batch
    batch_xs, _ = get_batch(data, i, batch_size)
    batch_xs = torch.Tensor(batch_xs)
    loss = model(batch_xs)
    loss.backward()
    opt.step()

  print(f"\nEpoch: {epoch}\tLoss: {loss}")


100%|██████████| 2/2 [00:00<00:00, 75.91it/s]


Epoch: 0	Loss: 74.44517517089844

Epoch: 1	Loss: 74.10515594482422

Epoch: 2	Loss: 73.7649154663086

Epoch: 3	Loss: 73.42491912841797

Epoch: 4	Loss: 73.08519744873047

Epoch: 5	Loss: 72.74502563476562

Epoch: 6	Loss: 72.40290069580078



100%|██████████| 2/2 [00:00<00:00, 82.75it/s]


Epoch: 7	Loss: 72.05651092529297

Epoch: 8	Loss: 71.70309448242188

Epoch: 9	Loss: 71.33988189697266
